### This notebook used to build, explore, and clean the keyword dataset
* Removal of non-ASCII kw
* Removal of offensive kw and URLs
* Identify and quintify duplicte records
* Dropping of all columns except kw

In [88]:
import pandas as pd
from typing import List, Dict, Optional
import inspect
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 100)

### Functions

In [89]:
def remove_non_ascii_kw(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes rows from a DataFrame where the "Keyword" column contains non-ASCII characters.
    
    Args:
        df (pandas.DataFrame): The DataFrame to process.
    
    Returns:
        pandas.DataFrame: The modified DataFrame with rows removed.
    """
    # Remove rows where "Keyword" contains non-ASCII characters
    df_start_len = len(df)
    df = df[df['kw'].apply(lambda x: all(ord(c) < 128 for c in x))]
    
    # Print some statistics
    print(f"df start length: {df_start_len}")
    print(f"df final length: {len(df)}")
    print(f"records removed: {df_start_len - len(df)}")
    
    # Return the modified DataFrame
    return df

In [90]:
def fix_nan(df: pd.DataFrame) -> pd.DataFrame:
    """
    Replaces NaN values in the "SERP features" and "CPC" columns of a pandas DataFrame with appropriate default values.
    
    Args:
        df (pandas.DataFrame): The DataFrame to process.
    
    Returns:
        pandas.DataFrame: The modified DataFrame with NaN values replaced.
    """
    # Count NaN values in "SERP features" and "CPC" columns
    serp_nan = df['SERP features'].isna().sum()
    cpc_nan = df['CPC'].isna().sum()
    
    # Replace NaN values with appropriate default values
    df['SERP features'] = df['SERP features'].fillna('none')
    df['CPC'] = df['CPC'].fillna(0)
    
    # Print some statistics
    print(f"SERP features `NaN` records replaced with `None`: {serp_nan}")
    print(f"CPC `NaN` records replaced with `0`: {cpc_nan}")
    
    # Return the modified DataFrame
    return df

In [91]:
def remove_blocklist_kw(df: pd.DataFrame, kw_blocklist: List[str]) -> pd.DataFrame:
    """
    Removes rows from a pandas DataFrame where the "Keyword" column contains any of the keywords in a blocklist.

    Args:
        df (pandas.DataFrame): The DataFrame to process.
        kw_blocklist (List[str]): A list of keywords to remove from the DataFrame.

    Returns:
        pandas.DataFrame: The modified DataFrame with the specified keywords removed.
    """
    # Get the length of the input DataFrame
    df_start_len = len(df)

    # Create a regular expression pattern from the blocklist
    kw_pattern = '|'.join(kw_blocklist)

    # Create a mask for rows containing keywords in the blocklist
    kw_mask = df['kw'].str.contains(kw_pattern, case=False)

    # Use the mask to remove the specified keywords from the DataFrame
    df = df[~kw_mask]

    # Print some statistics
    print(f"df start length: {df_start_len}")
    print(f"df final length: {len(df)}")
    print(f"records removed: {df_start_len - len(df)}")

    # Return the modified DataFrame
    return df

In [92]:
def remove_blocklist_url(df: pd.DataFrame, url_blocklist: List[str]) -> pd.DataFrame:
    """
    Removes rows from a pandas DataFrame where the "URL" column contains any of the URLs in a blocklist.

    Args:
        df (pandas.DataFrame): The DataFrame to process.
        url_blocklist (List[str]): A list of URLs to remove from the DataFrame.

    Returns:
        pandas.DataFrame: The modified DataFrame with the specified URLs removed.
    """
    # Get the length of the input DataFrame
    df_start_len = len(df)

    # Create a regular expression pattern from the blocklist
    url_pattern = '|'.join(url_blocklist)

    # Create a mask for rows containing URLs in the blocklist
    url_mask = df['url'].str.contains(url_pattern, case=False)

    # Use the mask to remove the specified URLs from the DataFrame
    df = df[~url_mask]

    # Print some statistics
    print(f"df start length: {df_start_len}")
    print(f"df final length: {len(df)}")
    print(f"records removed: {df_start_len - len(df)}")

    # Return the modified DataFrame
    return df

In [93]:
def show_df_info(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prints information about a DataFrame, including column names, data types, and non-null counts.
    
    Args:
        df (pandas.DataFrame): The DataFrame to process.
        
    Returns:
        pandas.DataFrame: A DataFrame containing the column names, data types, and non-null counts, and null counts of the input DataFrame.
    """
    col_names = df.columns.to_list()
    col_dtypes = df.dtypes.to_list()
    non_null_counts = df.count().to_list()
    null_counts = df.isnull().sum().to_list()
    info_df = pd.DataFrame({'column_name': col_names, 'dtype': col_dtypes, 'non_null_count': non_null_counts, 'null_count': null_counts})

    caller_frame = inspect.currentframe().f_back
    df_name = [var_name for var_name, var_val in caller_frame.f_locals.items() if var_val is df][0]

    print(f"DataFrame '{df_name}' has {len(df)} rows and {len(df.columns)} columns.")
    print("Here is a summary of the column names, data types and null counts:")
    return info_df

In [94]:
def find_duplicate_uid(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a DataFrame, finds and returns a new DataFrame containing only rows with duplicate 'uid' values.

    Parameters:
    df : pandas.DataFrame Input DataFrame with at least one column named 'uid'.

    Returns:
    pandas.DataFrame A new DataFrame containing only rows with duplicate 'uid' values.
    """
    # Get a boolean Series indicating which rows have a duplicate uid
    duplicates_mask = df.duplicated(subset=['uid'], keep=False)
    
    # Use the boolean mask to select the rows with duplicate uids
    duplicate_df = df[duplicates_mask]
    
    # Return the new DataFrame with only the duplicate rows
    return duplicate_df

In [95]:
def check_for_duplicate_uid(df: pd.DataFrame,) -> bool:
    test = df.duplicated(subset=['uid']).any()
    if test is True:
        print('*** Dataset has duplicate uid ***')
    else:
        print('Dataset has no duplicate uid')
    return test

In [96]:
def group_by_url_count(df: pd.DataFrame) -> pd.DataFrame:
    """
    Groups the input DataFrame by the count of each unique URL, sorts the resulting DataFrame by 'url_count' in descending order, and sets 'url' as the index column.

    Parameters:
    df : pandas.DataFrame A DataFrame containing at least one column named 'url' that contains URLs.

    Returns:
    pandas.DataFrame A DataFrame with unique URLs as the index column and the count of each URL as a column named 'url_count'. The DataFrame is sorted in descending order by 'url_count'.
    """
    
    # Group the DataFrame by the count of each unique URL
    grouped_df = df.groupby('url').size().reset_index(name='url_count')
    
    # Sort the DataFrame by 'url_count' in descending order
    sorted_df = grouped_df.sort_values(by='url_count', ascending=False)
    
    # Reset the index and make 'url' the index column
    sorted_df = sorted_df.set_index('url').reset_index()
    
    # Return the sorted DataFrame
    return sorted_df

In [97]:
def group_by_kw_count(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function takes a pandas DataFrame as input and groups it by the count of each unique value in the 'kw' column. It then sorts the resulting DataFrame by the count of each unique value in descending order and resets the index, making the 'kw' column the index. Finally, it returns the sorted DataFrame.

    Parameters:
    df: a pandas DataFrame with a 'kw' column.
    
    Returns:
    sorted_df: a pandas DataFrame sorted by the count of each unique value in the 'kw' column in descending order, with 'kw' as the index.

    Example usage:
    # Create a sample DataFrame
    df = pd.DataFrame({
        'kw': ['apple', 'banana', 'apple', 'cherry', 'banana', 'banana'],
        'count': [1, 2, 3, 4, 5, 6]
    })

    # Call the function to group by keyword count
    sorted_df = group_by_kw_count(df)

    # Print the resulting DataFrame
    print(sorted_df)
    
    This would output:

        kw      kw_count
    0  banana         3
    1   apple         2
    2  cherry         1
    """

    # Group the DataFrame by the count of each unique URL
    grouped_df = df.groupby('kw').size().reset_index(name='kw_count')
    
    # Sort the DataFrame by 'url_count' in descending order
    sorted_df = grouped_df.sort_values(by='kw_count', ascending=False)
    
    # Reset the index and make 'url' the index column
    sorted_df = sorted_df.set_index('kw').reset_index()
    
    # Return the sorted DataFrame
    return sorted_df

In [98]:
def add_count_repeats(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Counts the number of unique values in the specified column of a pandas DataFrame
    and adds a new column "counts_" + column to the DataFrame with the counts.
    
    Args:
        df (pandas.DataFrame): The DataFrame to operate on.
        column (str): The name of the column to count the unique values in.
        
    Returns:
        pandas.DataFrame: The modified DataFrame with the new count column.
    """
    # Count the number of unique values in the specified column
    counts = df[column].value_counts()
    
    # Create a new column with the counts
    new_col_name = "dup_count_" + column
    df[new_col_name] = df[column].map(counts)
    
    return df

In [99]:
def summarize_duplicates(df: pd.DataFrame, colums: List[str]) -> pd.DataFrame:
    """
    Counts the number of unique values, non-duplicate values (where count == 1), and duplicate values (where count > 1) in the specified columns of a pandas DataFrame as well as the minimum and maximum counts of values that are repeated.

    Args:
        df (pandas.DataFrame): The DataFrame to operate on.
        column_names (List[str]): A list of column names to summarize.
        
    Returns:
        pandas.DataFrame: A DataFrame summarizing counts of the number of unique values, non-duplicate values (where count == 1), and duplicate values (where count > 1) in the specified columns of a pandas DataFrame as well as the minimum and maximum counts of values that are repeated.
    """
    # Create an empty list to store the summary DataFrames
    summary = []
    
    # Iterate over the specified columns
    for column in colums:
        # Count the duplicates and non-duplicates in the column
        dup_counts = df[column].duplicated(keep=False)
        count_non_dup = (~dup_counts).sum()
        count_dup = dup_counts.sum()
        count_unique_dup = (dup_counts & (~df[column].duplicated())).sum()
        count_min_dup = df[column].duplicated(keep=False).groupby(df[column]).sum().loc[lambda x: x > 1].min()
        count_max_dup = df[column].duplicated(keep=False).groupby(df[column]).sum().max()
        
        # Count the unique values in the column
        count_unique = df[column].nunique()
        
        # Add a row to the summary DataFrame with the counts
        summary.append(pd.DataFrame({
            "column_name": [column],
            "count_unique": [count_unique],
            "count_non_dup": [count_non_dup],
            "count_dup": [count_dup],
            "count_unique_dup": [count_unique_dup],
            "count_min_dup": [count_min_dup if count_dup > 0 else 0],
            "count_max_dup": [count_max_dup],
        }))
    
    print(f"number of records: {len(df)}")
    # Combine the summary DataFrames into a single DataFrame
    summary = pd.concat(summary, ignore_index=True)
    
    return summary

### Code Execution

In [100]:
df1 = pd.read_csv('data/webmd.csv')
df2 = pd.read_csv('data/thespruce.csv')
df3 = pd.read_csv('data/amazon.csv')
df = pd.concat([df1, df2, df3])
df = df.reset_index(drop=True)
df = df.copy(deep=True)
drop_columns = ['Current position', 'Current URL inside', 'Updated']
df = df.drop(drop_columns, axis=1)
df = df.rename(columns={'Current URL': 'url'})
df = df.rename(columns={'Keyword': 'kw'})
df = df.copy(deep=True)

In [101]:
df.head(1)

,kw,SERP features,Volume,KD,CPC,Traffic,url
0,horny goat weed,Knowledge panel,76000,61,0.37,70591,https://www.webmd.com/vitamins/ai/ingredientmo...


In [102]:
show_df_info(df)

DataFrame 'df' has 90000 rows and 7 columns.
Here is a summary of the column names, data types and null counts:


,column_name,dtype,non_null_count,null_count
0,kw,object,90000,0
1,SERP features,object,89313,687
2,Volume,int64,90000,0
3,KD,int64,90000,0
4,CPC,float64,76574,13426
5,Traffic,int64,90000,0
6,url,object,90000,0


In [103]:
summarize_duplicates(df, ['url', 'kw'])

number of records: 90000


,column_name,count_unique,count_non_dup,count_dup,count_unique_dup,count_min_dup,count_max_dup
0,url,41220,27397,62603,13823,2,118
1,kw,88947,87895,2105,1052,2,3


In [104]:
url_counts = group_by_url_count(df)
url_counts.head()

,url,url_count
0,https://www.webmd.com/covid/coronavirus-incuba...,118
1,https://www.thespruce.com/budget-friendly-kids...,84
2,https://www.webmd.com/skin-problems-and-treatm...,79
3,https://www.thespruce.com/best-outdoor-solar-l...,79
4,https://www.webmd.com/allergies/ss/slideshow-p...,68


In [105]:
df = remove_non_ascii_kw(df)

df start length: 90000
df final length: 89917
records removed: 83


In [106]:
kw_block_list = ["penis", "vagina", "sex", "slut", "dick", "fuck", "milf", "cum ", "cunt", "rape", "porn", "boner", "cock", "whore", "bitch", "futa", "wet dreams", "cowgirl position",  "reverse cowgirl", "girl squirt", "girls squirt", "squirting-orgasm", " squirt pee", "girl's body", "dildo", "butt-plug", "good head", "smells bad down there", "xxx", "wormwood", "fetish",  "vibrator", "sandalias de mujer"]

In [107]:
df = remove_blocklist_kw(df, kw_block_list)

df start length: 89917
df final length: 88842
records removed: 1075


In [108]:
url_blocklist = ["aws.amazon", "www.amazon.com/kindle-dbs", "squirting-orgasm"]

In [109]:
df = remove_blocklist_url(df, url_blocklist)

df start length: 88842
df final length: 88704
records removed: 138


In [110]:
df = fix_nan(df)

SERP features `NaN` records replaced with `None`: 602
CPC `NaN` records replaced with `0`: 13180


In [111]:
df.head(1)

,kw,SERP features,Volume,KD,CPC,Traffic,url
0,horny goat weed,Knowledge panel,76000,61,0.37,70591,https://www.webmd.com/vitamins/ai/ingredientmo...


In [112]:
show_df_info(df)

DataFrame 'df' has 88704 rows and 7 columns.
Here is a summary of the column names, data types and null counts:


,column_name,dtype,non_null_count,null_count
0,kw,object,88704,0
1,SERP features,object,88704,0
2,Volume,int64,88704,0
3,KD,int64,88704,0
4,CPC,float64,88704,0
5,Traffic,int64,88704,0
6,url,object,88704,0


In [113]:
summarize_duplicates(df, ['url', 'kw'])

number of records: 88704


,column_name,count_unique,count_non_dup,count_dup,count_unique_dup,count_min_dup,count_max_dup
0,url,40750,27114,61590,13636,2,118
1,kw,87664,86625,2079,1039,2,3


In [116]:
df.to_csv('data/combined.csv', index=False)

In [117]:
df_10_000 = df.sample(n=10000, random_state=42)
df_10_000.to_csv('data/kw_10_000.csv', index=False)

In [118]:
df_1_000 = df_10_000.sample(n=1000, random_state=42)
df_1_000.to_csv('data/kw_1_000.csv', index=False)

In [119]:
summarize_duplicates(df_1_000, ['kw', 'url'])

number of records: 1000


,column_name,count_unique,count_non_dup,count_dup,count_unique_dup,count_min_dup,count_max_dup
0,kw,1000,1000,0,0,0,0
1,url,966,934,66,32,2,3


In [120]:
df_1_000.nunique()

kw               1000
SERP features     413
Volume            154
KD                 90
CPC               298
Traffic           723
url               966
dtype: int64

In [121]:
df_1_000 = df_1_000.drop(['SERP features', 'Volume', 'KD', 'CPC', 'Traffic', 'url'], axis=1)
df_1_000

,kw
48569,stone for fireplace
46767,country french decor
14576,infant fever
71185,uglies
22373,why are my ears ringing
...,...
58572,underbed storage drawers
63385,weight scale
78884,yperx cloud core
55310,magnolia kobus


In [122]:
df_1_000 = df_10_000.sample(n=1000, random_state=42)
df_1_000.to_csv('data/kw_1_000.csv', index=False)